# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 11:35AM,241203,10,8239056,Eywa Pharma Inc.,Released
1,Jul 6 2022 11:35AM,241203,10,8239057,Eywa Pharma Inc.,Released
2,Jul 6 2022 11:35AM,241203,10,8239058,Eywa Pharma Inc.,Released
3,Jul 6 2022 11:35AM,241203,10,8239105,Eywa Pharma Inc.,Released
4,Jul 6 2022 11:35AM,241203,10,8239107,Eywa Pharma Inc.,Released
5,Jul 6 2022 11:35AM,241203,10,8239109,Eywa Pharma Inc.,Released
6,Jul 6 2022 11:35AM,241203,10,8239110,Eywa Pharma Inc.,Released
7,Jul 6 2022 11:35AM,241203,10,8239112,Eywa Pharma Inc.,Released
8,Jul 6 2022 11:35AM,241203,10,8239159,Eywa Pharma Inc.,Released
9,Jul 6 2022 11:34AM,241205,10,8239102,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
65,241200,Released,1
66,241201,Released,13
67,241203,Released,9
68,241204,Released,4
69,241205,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241200,NaN,NaN,1.0
241201,NaN,NaN,13.0
241203,NaN,NaN,9.0
241204,NaN,NaN,4.0
241205,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241095,0.0,0.0,8.0
241100,0.0,0.0,1.0
241101,0.0,3.0,1.0
241103,0.0,4.0,11.0
241104,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241095,0,0,8
241100,0,0,1
241101,0,3,1
241103,0,4,11
241104,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241095,0,0,8
1,241100,0,0,1
2,241101,0,3,1
3,241103,0,4,11
4,241104,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241095,,,8
1,241100,,,1
2,241101,,3,1
3,241103,,4,11
4,241104,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 11:35AM,241203,10,Eywa Pharma Inc.
9,Jul 6 2022 11:34AM,241205,10,Beach Products Inc
10,Jul 6 2022 11:34AM,241204,10,Aerogen LTD.
14,Jul 6 2022 11:21AM,241201,20,"ACI Healthcare USA, Inc."
27,Jul 6 2022 11:13AM,241200,10,"Methapharm, Inc."
28,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC"
33,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC"
37,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc"
38,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC"
58,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 11:35AM,241203,10,Eywa Pharma Inc.,,,9
1,Jul 6 2022 11:34AM,241205,10,Beach Products Inc,,,1
2,Jul 6 2022 11:34AM,241204,10,Aerogen LTD.,,,4
3,Jul 6 2022 11:21AM,241201,20,"ACI Healthcare USA, Inc.",,,13
4,Jul 6 2022 11:13AM,241200,10,"Methapharm, Inc.",,,1
5,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",,,5
6,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",,,4
7,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",,,1
8,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",,13,7
9,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,,,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 11:35AM,241203,10,Eywa Pharma Inc.,9,,
1,Jul 6 2022 11:34AM,241205,10,Beach Products Inc,1,,
2,Jul 6 2022 11:34AM,241204,10,Aerogen LTD.,4,,
3,Jul 6 2022 11:21AM,241201,20,"ACI Healthcare USA, Inc.",13,,
4,Jul 6 2022 11:13AM,241200,10,"Methapharm, Inc.",1,,
5,Jul 6 2022 10:53AM,241197,15,"Carwin Pharmaceutical Associates, LLC",5,,
6,Jul 6 2022 10:52AM,241196,15,"Laser Pharmaceuticals, LLC",4,,
7,Jul 6 2022 10:52AM,241198,19,"Avet Pharmaceuticals Labs, Inc",1,,
8,Jul 6 2022 10:42AM,241193,15,"Brookfield Pharmaceuticals, LLC",7,13,
9,Jul 6 2022 10:40AM,241192,10,Eywa Pharma Inc.,17,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 11:35AM,241203,10,Eywa Pharma Inc.,9,,
1,Jul 6 2022 11:34AM,241205,10,Beach Products Inc,1,,
2,Jul 6 2022 11:34AM,241204,10,Aerogen LTD.,4,,
3,Jul 6 2022 11:21AM,241201,20,"ACI Healthcare USA, Inc.",13,,
4,Jul 6 2022 11:13AM,241200,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 11:35AM,241203,10,Eywa Pharma Inc.,9.0,NaN,NaN
1,Jul 6 2022 11:34AM,241205,10,Beach Products Inc,1.0,NaN,NaN
2,Jul 6 2022 11:34AM,241204,10,Aerogen LTD.,4.0,NaN,NaN
3,Jul 6 2022 11:21AM,241201,20,"ACI Healthcare USA, Inc.",13.0,NaN,NaN
4,Jul 6 2022 11:13AM,241200,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 11:35AM,241203,10,Eywa Pharma Inc.,9.0,0.0,0.0
1,Jul 6 2022 11:34AM,241205,10,Beach Products Inc,1.0,0.0,0.0
2,Jul 6 2022 11:34AM,241204,10,Aerogen LTD.,4.0,0.0,0.0
3,Jul 6 2022 11:21AM,241201,20,"ACI Healthcare USA, Inc.",13.0,0.0,0.0
4,Jul 6 2022 11:13AM,241200,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8199534,465.0,1.0,3.0
12,964560,24.0,5.0,0.0
15,964776,40.0,26.0,0.0
16,1687865,7.0,9.0,0.0
19,964587,58.0,34.0,13.0
20,964591,92.0,11.0,0.0
21,241183,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8199534,465.0,1.0,3.0
1,12,964560,24.0,5.0,0.0
2,15,964776,40.0,26.0,0.0
3,16,1687865,7.0,9.0,0.0
4,19,964587,58.0,34.0,13.0
5,20,964591,92.0,11.0,0.0
6,21,241183,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,465.0,1.0,3.0
1,12,24.0,5.0,0.0
2,15,40.0,26.0,0.0
3,16,7.0,9.0,0.0
4,19,58.0,34.0,13.0
5,20,92.0,11.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,465.0
1,12,Released,24.0
2,15,Released,40.0
3,16,Released,7.0
4,19,Released,58.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,13.0,0.0,0.0
Executing,1.0,5.0,26.0,9.0,34.0,11.0,0.0
Released,465.0,24.0,40.0,7.0,58.0,92.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,13.0,0.0,0.0
1,Executing,1.0,5.0,26.0,9.0,34.0,11.0,0.0
2,Released,465.0,24.0,40.0,7.0,58.0,92.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,13.0,0.0,0.0
1,Executing,1.0,5.0,26.0,9.0,34.0,11.0,0.0
2,Released,465.0,24.0,40.0,7.0,58.0,92.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()